In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/mini-model-features/"))

['test_features_c.pkl', 'test_features_w.pkl', 'train_features_h.pkl', 'test_features_h.pkl', 'train_features_r.pkl', 'train_features_c.pkl', 'train_features_s.pkl', 'test_features_r.pkl', 'train_features_w.pkl', 'test_features_s.pkl']


In [2]:
train = pd.read_pickle('../input/mini-model-features/train_features_w.pkl')
test= pd.read_pickle('../input/mini-model-features/test_features_w.pkl')

train_y = train['isFraud'].copy()

print(train.shape)
print(train_y.shape)
print(test.shape)

(439670, 214)
(439670,)
(360987, 214)


In [3]:
rm_cols = ['isFraud', 'TransactionDT', 'DT','addr2', 'C12', 'TransactionAmt','V1','V33','V34','V41',
          'V51','V52','V73','V74','V88','V94','V107','V303','V304','V305','uid','uid2','uid3','uid4'
           'D1','D3','D4','D10','D11','D15', 
          ]
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)

In [4]:
train[features_columns].head()

,dist1,D1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V35,V36,V37,V38,V44,V45,V46,V47,V48,V49,V53,V54,V55,V56,V61,V62,V66,V67,V69,V70,V75,V76,V77,V78,V82,V83,V86,V87,V90,V91,V96,V98,V101,V104,V105,V106,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V279,V280,V283,V284,V285,V287,V290,V291,V292,V293,V294,V295,V297,V298,V299,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,M_sum,M_na,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,uid4_TransactionAmt_std,DT_month,D1_day_mean,D1_week_mean,D3_day_mean,D3_week_mean,D4_day_mean,D4_week_mean,D10_day_mean,D10_week_mean,D11_day_mean,D11_week_mean,D15_day_mean,D15_week_mean,addr1_target_code,addr1_fq_enc,C1_target_code,C1_fq_enc,C2_target_code,C2_fq_enc,C5_target_code,C5_fq_enc,C6_target_code,C6_fq_enc,C9_target_code,C9_fq_enc,C11_target_code,C11_fq_enc,C13_target_code,C13_fq_enc,C14_target_code,C14_fq_enc,M4_target_code,M4_fq_enc,card1_target_code,card1_fq_enc,card2_target_code,card2_fq_enc,card3_target_code,card4_target_code,card5_target_code,card5_fq_enc,card6_target_code,card6_fq_enc,M1_target_code,M1_fq_enc,M2_target_code,M2_fq_enc,M3_target_code,M3_fq_enc,M5_target_code,M5_fq_enc,M6_target_code,M6_fq_enc,M7_target_code,M7_fq_enc,M8_target_code,M8_fq_enc,M9_target_code,M9_fq_enc,P_emaildomain_target_code,cents_target_code,cents_fq_enc,DT_hour_target_code,DT_hour_fq_enc,DT_day_week_target_code,DT_day_week_fq_enc,DT_day_target_code,DT_day_fq_enc
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,19.0,14.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,4,3,1,425.7500,999.0000,NaN,NaN,153.625,191.5000,999.000,258.0000,210.7500,258.0000,210.7500,193.0000,176.12500,203.375,12,-106.9375,-104.75,-14.6875,-13.84375,NaN,NaN,-134.25,-130.625,-142.125,-148.5,-209.875,-214.75,0.011927,37470,0.022501,375774,0.019654,394000,0.032130,410437,0.017961,404235,0.019002,420354,0.025737,134441,0.021306,193159,0.022447,392631,0.055653,7493,0.020391,33,0.020303,14101,0.020521,0.077283,0.042353,284,0.042874,123746,0.019897,649436,0.018025,588323,0.016832,518244,0.026328,240155,0.017218,349499,0.020561,284709,0.020328,284682,0.020393,284682,0.020384,0.020893,2383,0.017338,51086,0.019310,132019,0.017174,32568
2987001,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,6,1,224.2500,999.0000,274.750,999.000,153.625,278.2500,999.000,224.8750,999.0000,224.8750,999.0000,244.1250,999.00000,999.000,12,-120.9375,-118.75,NaN,NaN,-183.0,-188.625,-147.25,-143.625,NaN,NaN,-209.875,-214.75,0.021384,66766,0.023170,375774,0.019697,394000,0.031957,410437,0.018123,404235,0.017315,44990,0.019174,506525,0.020975,193159,0.022587,392631,0.032515,338042,0.046987,1222,0.031769,3963,0.020387,0.017559

In [5]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':1000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'early_stopping_rounds':100, 
                } 

In [6]:
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  358122 81548
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[904]	training's auc: 1	valid_1's auc: 0.923255
Fold: 1  -  363531 76139
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[889]	training's auc: 0.999992	valid_1's auc: 0.901467
Fold: 2  -  364760 74910
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[998]	training's auc: 1	valid_1's auc: 0.927373
Fold: 3  -  369590 70080
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[997]	training's auc: 1	valid_1's auc: 0.913835
Fold: 4  -  370970 68700
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[995]	training's auc: 1	valid_1's auc: 0.928147
Fold: 5  -  371377 68293
Training until validation scores don't improve for 100 rounds

In [7]:
test_predictions.to_pickle('../input/mini-model-predictions/predictions_w.pkl') #0.9231690036072205